## Populate Task

In [10]:
from dotenv import load_dotenv
import json

In [11]:
PATH_FILE_DATA = 'C:/Users/ADMIN/Desktop/DATN/Extract_Information/data_crawl/data/data_crawl.jsonl'
PATH_FILE_TEMPLATE = 'C:/Users/ADMIN/Desktop/DATN/Extract_Information/tasks/task_template.json'

In [12]:
task = 'generated_knowledge_one_shot'

In [13]:
category_attribute_2_human = {
    'Laptops_Processor Brand':['Dell Inspiron 14z i14z-2100sLV 14.0-Inch Ultrabook (1.9 GHz Intel Core i3-3227U Processor, 6GB DDR3, 500GB HDD, Windows 8) Moon Silver [Discontinued By Manufacturer]', 'HP NW9440 17.0 Inch Laptop, Intel Core 2 Duo T7400 2.16 GHz, 2 GB DDR2 SDRAM, 100 GB Hard Drive, Genuine Windows XP Professional (SP2)', 'OQO 1070103-US Model 02 5\" Ultra Mobile PC UMPC'],
    'Laptops_Processor Speed':['HP Pavilion dv6113us 15.4\" Notebook Laptop PC (AMD Turion 64 X2 Dual-Core TL-50 1.6GHz, 1GB RAM, 80GB HDD / DVD\u00b1RW DL / Wireless / WXP Media Center) by CE', 'Dell Inspiron 14z i14z-2100sLV 14.0-Inch Ultrabook (1.9 GHz Intel Core i3-3227U Processor, 6GB DDR3, 500GB HDD, Windows 8) Moon Silver [Discontinued By Manufacturer]', 'Averatec AV3255P1-01 Laptop (AMD Athlon XP-M 2000+, 512 MB RAM, 40 GB Hard drive, 802.11g, DVD+CD-RW combo)'],
    'Laptops_Screen Size':['HP Pavilion dv6113us 15.4\" Notebook Laptop PC (AMD Turion 64 X2 Dual-Core TL-50 1.6GHz, 1GB RAM, 80GB HDD / DVD\u00b1RW DL / Wireless / WXP Media Center) by CE', 'Acer Aspire S7 13.3-Inch Touchscreen Ultrabook (Intel Core i5-4200U Processor up to 2.6GHz, 8GB DDR3 Memory, 256GB SSD, FHD 1080P Touchscreen Display, Backlit Keyboard, 802.11AC WiFi, Windows 8.1, 0.5 inch thin and 2.9 Lbs) Crystal White', 'Toshiba A45-S121 Laptop (2.8 GHz Celeron, 256 MB RAM, 40 GB Hard Drive, DVD/CD-RW Combo)'],
}

category_attribute_2_system = {
    'Laptops_Processor Brand': ['Intel', 'Intel', 'I do not know.'],
    'Laptops_Processor Speed':['1.6GHz', '1.9 GHz', 'I do not know.'],
    'Laptops_Screen Size':['15.4 inches', '13.3 inches', 'I do not know.'],
}

# Update Task Dict
def update_task_dict_from_file(file_name, task_dict):
    with open(file_name, 'r') as f:
        for line in f:
            record = json.loads(line)
            
            example = {}
            example['human_0'] = category_attribute_2_human['Laptops_Screen Size'][0]
            example['system_0'] = category_attribute_2_system['Laptops_Screen Size'][0]
            example['attribute'] = "Screen Size"
            example['input'] = '{} \n Requested Attribute: Screen Size'.format(record['title'])
            task_dict['examples'].append(example)
            
            example1 = {}
            example1['human_0'] = category_attribute_2_human['Laptops_Processor Brand'][0]
            example1['system_0'] = category_attribute_2_system['Laptops_Processor Brand'][0]
            example1['attribute'] = "Processor Brand"
            example1['input'] = '{} \n Requested Attribute: Processor Brand'.format(record['title'])
            task_dict['examples'].append(example1)
            
            example2 = {}
            example2['human_0'] = category_attribute_2_human['Laptops_Processor Speed'][0]
            example2['system_0'] = category_attribute_2_system['Laptops_Processor Speed'][0]
            example2['attribute'] = "Processor Speed"
            example2['input'] = '{} \n Requested Attribute: Processor Speed'.format(record['title'])
            task_dict['examples'].append(example2)

    return task_dict

In [14]:
# Load OPENAI_API_KEY from .env file
load_dotenv()

True

In [15]:
# Load task template
with open(PATH_FILE_TEMPLATE, 'r') as f:
    task_dict = json.load(f)

task_dict['task_prefix'] = "Extract all attribute-value pairs from the product title. Afterwards, return only the exact attribute value of the requested attribute. Your answer must only contain the attribute value. If the attribute is not applicable or the attribute value cannot be derived from the product title, respond with 'I do not know.'."

# Generate examples
task_dict = update_task_dict_from_file(PATH_FILE_DATA, task_dict)

with open('task_data_crawl.json', 'w', encoding='utf-8') as f:
    json.dump(task_dict, f, indent=4)

## Evaluate Task

In [16]:
# Initialize processing
from tqdm import tqdm
from langchain import LLMChain
from langchain.chat_models import ChatOpenAI

from langchain.prompts import (
    ChatPromptTemplate,
    SystemMessagePromptTemplate,
    HumanMessagePromptTemplate,
    AIMessagePromptTemplate
)

# Put task prefix into system message.
prompt_list = []
system_message_prompt = SystemMessagePromptTemplate.from_template(task_dict['task_prefix'])
prompt_list.append(system_message_prompt)

# Prepare one-shot  
human_example_template="{human_msg_0}"
human_example_message_prompt = HumanMessagePromptTemplate.from_template(human_example_template)
prompt_list.append(human_example_message_prompt)

ai_example_template="{ai_msg_0}"
ai_example_message_prompt = AIMessagePromptTemplate.from_template(ai_example_template)
prompt_list.append(ai_example_message_prompt)

prompt_list.append(system_message_prompt)

# Add 
human_template="{input_string}"
human_message_prompt = HumanMessagePromptTemplate.from_template(human_template)
prompt_list.append(human_message_prompt)

chat_prompt = ChatPromptTemplate.from_messages(prompt_list)

gpt_3_turbo = ChatOpenAI(model_name='gpt-3.5-turbo-0125', temperature=0)

llm_chain = LLMChain(
    prompt=chat_prompt,
    llm=gpt_3_turbo
)

# Save model
task_dict['model'] = llm_chain.llm.model_name

In [19]:
def select_and_run_llm(attribute, human, system, input):
        pred = None
        if len(input) == 0:
            # No input text provided.
            return pred
        
        response = llm_chain.run(input_string = input, human_msg_0=human, ai_msg_0=system)
        print(response)
        try:
            pred ={}
            pred[attribute] = response
        except Exception as e:
            print(e)
            print('Response: ')
            print(response)
        return pred

In [20]:
# General task prefix
from tqdm import tqdm
from langchain.callbacks import get_openai_callback

with get_openai_callback() as cb:
    preds = [select_and_run_llm(example['attribute'], example['human_0'], example['system_0'], example['input']) for example in tqdm(task_dict['examples'])]
    
    task_dict['total_tokens'] = cb.total_tokens
    print(f"Total Tokens: {cb.total_tokens}")

  0%|          | 1/1341 [00:01<23:06,  1.03s/it]

15.6 inches


  0%|          | 2/1341 [00:01<16:48,  1.33it/s]

Intel


  0%|          | 3/1341 [00:02<16:33,  1.35it/s]

...

14 inches


100%|█████████▉| 1340/1341 [14:54<00:00,  1.69it/s]

AMD


100%|██████████| 1341/1341 [14:54<00:00,  1.50it/s]

I do not know.
Total Tokens: 333389


In [21]:
print(preds)

[{'Screen Size': '15.6 inches'}, {'Processor Brand': 'Intel'}, {'Processor Speed': '1.2GHz'}, {'Screen Size': '14 inches'}, {'Processor Brand': 'Intel'}, {'Processor Speed': '1.2GHz'}, {'Screen Size': '15.6 inches'}, {'Processor Brand': 'Intel'}, {'Processor Speed': '1.7GHz'}, {'Screen Size': '14 inches'}, {'Processor Brand': 'AMD'}, {'Processor Speed': '2.8GHz'}, {'Screen Size': '14 inches'}, {'Processor Brand': 'Intel'}, {'Processor Speed': '1.3GHz'}, {'Screen Size': '14 inches'}, {'Processor Brand': 'Intel'}, {'Processor Speed': '1.3GHz'}, {'Screen Size': '15.6 inches'}, {'Processor Brand': 'Intel'}, {'Processor Speed': '1.3GHz'}, {'Screen Size': '15.6 inches'}, {'Processor Brand': 'Intel'}, {'Processor Speed': '1.7GHz'}, {'Screen Size': '15.6 inches'}, {'Processor Brand': 'Intel'}, {'Processor Speed': '1.7GHz'}, {'Screen Size': '14 inches'}, {'Processor Brand': 'Intel'}, {'Processor Speed': '1.3GHz'}, {'Screen Size': '15.6 inches'}, {'Processor Brand': 'Intel'}, {'Processor Speed':

In [22]:
# Tên tệp JSON bạn muốn lưu
file_name = "result_open.json"

# Ghi từ điển vào tệp JSON
with open(file_name, 'w') as json_file:
    json.dump(preds, json_file)

In [26]:
i = 0
data=[]
with open(PATH_FILE_DATA, 'r') as f:
    for line in f:
        record = json.loads(line)
            
        example = {}
        example['title'] = record['title']
        example['Screen Size'] = preds[i]['Screen Size']
        example['Processor Brand'] = preds[i+1]['Processor Brand']
        example['Processor Speed'] = preds[i+2]['Processor Speed']
        i = i + 3
        data.append(example)

In [27]:
with open('C:/Users/ADMIN/Desktop/DATN/Extract_Information/data_crawl/results/results_generated_datacrawl.json', 'w', encoding='utf-8') as json_file:
    json.dump(data, json_file)

In [28]:
import pandas as pd
# Tạo DataFrame từ danh sách
df = pd.DataFrame(data)

In [29]:
df.to_csv('extract_gpt_generated.csv', index=False, encoding='utf-8')

In [2]:
import json
with open('C:/Users/ADMIN/Desktop/DATN/Extract_Information/data_crawl/results/results_generated_datacrawl.json', 'r', encoding='utf-8') as json_file:
    data = json.load(json_file)
  
data_compare = []  
with open('C:/Users/ADMIN/Desktop/DATN/Extract_Information/data_crawl/data/data_crawl.jsonl', 'r') as f:
    for line in f:
        product = json.loads(line)  
        extract_product = {}
        extract_product['title'] = product['title']
        extract_product['Screen Size'] = product['Screen Size']
        extract_product['Processor Brand'] = product['Processor Brand']
        extract_product['Processor Speed'] = product['Processor Speed']
        data_compare.append(extract_product)

print(data)
print(data_compare)
        
# for dt, data_compare in zip(data, data_compare):

[{'title': 'HP 15s fq5229TU i3 1215U (8U237PA) (Intel UHD, 15.6 inches, 1.2GHz)', 'Screen Size': '15.6 inches', 'Processor Brand': 'Intel', 'Processor Speed': '1.2GHz'}, {'title': 'HP 240 G9 i3 1215U (6L1X7PA) (Intel UHD, 14 inches, 1.2GHz)', 'Screen Size': '14 inches', 'Processor Brand': 'Intel', 'Processor Speed': '1.2GHz'}, {'title': 'HP Pavilion 15 eg2081TU i5 1240P (7C0Q4PA) (Intel Iris Xe, 15.6 inches, 1.7GHz)', 'Screen Size': '15.6 inches', 'Processor Brand': 'Intel', 'Processor Speed': '1.7GHz'}, {'title': 'HP 245 G10 R5 7520U (8F155PA) (Radeon, 14 inches, 2.8GHz)', 'Screen Size': '14 inches', 'Processor Brand': 'AMD', 'Processor Speed': '2.8GHz'}, {'title': 'HP Pavilion 14 dv2073TU i5 1235U (7C0P2PA) (Intel Iris Xe, 14 inches, 1.3GHz)', 'Screen Size': '14 inches', 'Processor Brand': 'Intel', 'Processor Speed': '1.3GHz'}, {'title': 'HP 240 G9 i5 1235U (6L1Y2PA) (Intel UHD, 14 inches, 1.3GHz)', 'Screen Size': '14 inches', 'Processor Brand': 'Intel', 'Processor Speed': '1.3GHz'},

In [3]:
eval_nn = 0 # the model can predict No value (I do not know.) when ground truth is No attribute value
eval_nv = 0 # some incorrect Value when ground truth is No attribute value
eval_vn = 0 # the model can predict No value (I do not know.) when the ground truth has attribute Values
eval_vc = 0 # Correct values when the ground truth has attribute Values (TP)
eval_vw = 0 # Wrong values when the ground truth has attribute Values (FN-FP)

for dt, dt_compare in zip(data,data_compare):
    if dt_compare['Screen Size'] == "I do not know.":
        if dt_compare['Screen Size'] == dt['Screen Size']:
            eval_nn += 1
        else:
            eval_nv += 1
    else:   
        if dt_compare['Screen Size'] == dt['Screen Size']:
            eval_vc += 1
        elif dt['Screen Size'] == "I do not know.":
            eval_vn += 1
        else:
            eval_vw += 1
            
    if dt_compare['Processor Brand'] == "I do not know.":
        if dt_compare['Processor Brand'] == dt['Processor Brand']:
            eval_nn += 1
        else:
            eval_nv += 1
    else:   
        if dt_compare['Processor Brand'] == dt['Processor Brand']:
            eval_vc += 1
        elif dt['Processor Brand'] == "I do not know.":
            eval_vn += 1
        else:
            eval_vw += 1
            
    if dt_compare['Processor Speed'] == "I do not know.":
        if dt_compare['Processor Speed'] == dt['Processor Speed']:
            eval_nn += 1
        else:
            eval_nv += 1
    else:   
        if dt_compare['Processor Speed'] == dt['Processor Speed']:
            eval_vc += 1
        elif dt['Processor Speed'] == "I do not know.":
            eval_vn += 1
        else:
            eval_vw += 1

precision = round((eval_vc / (eval_nv + eval_vc + eval_vw))*100, 2) if (eval_nv + eval_vc + eval_vw) > 0 else 0 
recall = round((eval_vc / (eval_vn + eval_vc + eval_vw))*100, 2) if (eval_vn + eval_vc + eval_vw) > 0 else 0
f1 = round(2* precision* recall/ (precision + recall), 2) if (precision + recall) > 0 else 0

print(f"Precision: {precision}")
print(f"Recall: {recall}")
print(f"F1: {f1}")                

Precision: 63.37
Recall: 62.2
F1: 62.78
